# Heat diffusion: using Landlab to model the temperature field around an Arctic talik

*(Greg Tucker, University of Colorado Boulder, May 2025)*

The Arctic is famous for its permafrost: ground that remains below 0$^\circ$C and frozen permanently, or at least for 2 years or more in a row. Yet there are exceptions to the permafrost rule. The ground beneath shallow lakes and rivers often forms what's known as a *talik*: an area of unfrozen ground that is held above the freezing point by virtue of its contact with seasonally unfrozen water at the surface.

The purpose of this notebook is to demonstrate how to create a 2D thermal model in Landlab, using as a motivating example a hypothetical talik. The model domain consists of a vertical slice of ground. The model calculates heat conduction through the slice, assuming an initial geothermal gradient. Geothermal heat flows in through the bottom boundary. The top boundary is split into two parts. One part, representing dry land, has a surface temperature that varies sinusoidally throughout each year. The other, representing a shallow lake, stays fixed at or near the freezing point. The model is highly simplified, representing only heat conduction, and omitting consideration of the heat taken in or released by phase changes (solid to liquid or the reverse).

## Mathematical theory

The equation governing 2D heat conduction in a solid, with no internal sources or sinks, is:

$$\frac{\partial T}{\partial t} = -\frac{1}{\rho C_p} \nabla \cdot \mathbf{q}$$

where $T$ is temperature, $t$ is time, $\rho$ is material density, and $C_p$ is the specific heat of the material (usually expressed in Joules per kilogram per degree). Here $\mathbf{q} = [q_x, q_y]$ is the two-dimensional vector of heat flux (power per unit area, e.g., Watts per square meter). The symbol $\nabla\cdot$ represents the divergence operator,  defined as

$$\nabla\cdot \equiv \frac{\partial}{\partial x}\hat{\mathbf{i}} + \frac{\partial}{\partial y}\hat{\mathbf{j}}$$

where $\hat{\mathbf{i}}$ and $\hat{\mathbf{j}}$ are unit vectors in the $x$ and $y$ directions, respectively.

The heat flux is given by Fourier's law:

$$\mathbf{q} = -k \nabla{T}$$

where $k$ is thermal conductivity (e.g., Watts per meter per Kelvin) and $\nabla{T}$ is the temperature gradient.

## Numerical implementation

This example uses the Finite Volume Method, in which an average value of $T$ is tracked in each cell within a grid, in this case a regular rectilinear grid with square cells of dimensions $\Delta x$ by $\Delta x$. In a Landlab grid, each cell contains a grid *node*. The temperature gradient between each adjacent pair of nodes is calculated by taking the difference between $T$ at the two nodes and dividing by the distance between them (in this case, that distance is always equal to the grid spacing $\Delta x$). Each adjacent pair of nodes is represented by a grid objected called a *link*: a directed line segment that extends from its *tail node* to its *head node*. Therefore, every value of temperature gradient calculated between nodes is assigned to the corresponding link. The sign of the gradient is taken in reference to the link's direction: if temperature at the head node is greater than at the tail node, then the gradient is positive; if the reverse, the gradient is negative, and if the two temperatures are equal, the gradient is of course zero.

The faces of each grid cell are associated with links. The heat flux per unit width across a cell face is calculated using Fourier's law, by multiplying the corresponding temperature gradient by $-k$. Once heat fluxes have been calculated for all of the cell faces, the *flux divergence* is calculated for each grid cell by multiplying each $q$ (in Watts per meter) by cell width and adding up the resulting total heat fluxes (now in Watts) coming in or out of the four cell faces. This total flux is then divided by cell area to get the flux divergence (in Watts per square meter).

For more examples of finite-volume numerical methods using Landlab, see:

- [Tutorial notebook on "Using Landlab’s gradient and flux divergence functions"](https://landlab.csdms.io/tutorials/gradient_and_divergence/gradient_and_divergence.html)

- [Tutorial notebook on "Introduction to Landlab: Creating a simple 2D scarp diffusion model"](https://landlab.csdms.io/tutorials/fault_scarp/landlab-fault-scarp.html)

For more on Landlab grids, see:

- [Grid object tutorial (https://landlab.csdms.io/tutorials/grids/grid_object_demo.html)](https://landlab.csdms.io/tutorials/grids/grid_object_demo.html)

- [Overview of different grid types (https://landlab.csdms.io/tutorials/grids/diverse_grid_classes.html)](https://landlab.csdms.io/tutorials/grids/diverse_grid_classes.html)

- [Hobley et al. (2017)](https://doi.org/10.5194/esurf-5-21-2017)

- [Barnhart et al. (2020)](https://doi.org/10.5194/esurf-8-379-2020)

## Code

First, we import stuff: numpy for numerical calculations, matplotlib's `pyplot` library for plotting, and Landlab's `RasterModelGrid` (which we'll use for the simulation grid) and the plotting function `imshow_grid`.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from landlab import RasterModelGrid, imshow_grid

Define parameters for the simulation (here using rather rough values, since this is just meant to be an illustration of how to build a model)::

In [ ]:
# Parameters
domain_width = 500.0  # width of domain, m
domain_depth = 320.0  # depth of domain, m
lake_width = 250.0  # width of lake, m
dx = 5.0  # grid cell spacing, m
conductivity = 2.5  # thermal conductivity, W/mK
density = 1500.0  # mass density of permafrost, kg/m3
specific_heat = 1000.0  # specific heat of permafrost, J/kg K
geothermal_heat_flux = 0.025  # heat flux at base of profile, W/m2
courant = 0.2  # Courant number for finite-volume solution
Ts_mean = 0.5 * (5.4 - 24.4)  # Mean surface temperature outside of lake, C
Ts_amp = 0.5 * (24.4 + 5.4)  # Amplitude of annual temperature variation, C
period = 365.25 * 24.0 * 3600.0  # Period of temperature variation, s
run_duration = 100.0 * 365.25 * 24.0 * 3600.0  # Duration of run, s
T_talik_base = 1.0  # temperature of lake base, C

# Derived
kappa = conductivity / (density * specific_heat)  # thermal diffusivity, m2/s
dt = courant * dx * dx / kappa  # time-step duration, s
dt = min(dt, 3600.0 * 24)  # time steps should be no longer than one day
nrows = int(domain_depth / dx) + 1  # number of node rows
ncols = int(domain_width / dx) + 1  # number of node columns
Tgrad_mean = geothermal_heat_flux / conductivity  # temperature gradient, C/m
one_over_rho_cp = 1.0 / (
    density * specific_heat
)  # 1 / rho Cp (for calculating rate of change of temperature)
current_time = 0.0  # current time in simulation, s
num_steps = int(run_duration / dt)  # number of time steps

Set up the grid and associated data:

- A Landlab `RasterModelGrid` object is used to represent the domain, a vertical slice through the ground
- Lateral boundaries are closed to the flow of heat
- An array of values for temperature, `T`, is defined for the grid nodes
- An array of values for heat flux, `q`, is defined for the grid *links*, which are directed line segments that connect each pair of adjacent nodes
- The temperature field is initialized to the equilibrium geothermal gradient
- An array of "lake nodes" is identified as a set of nodes across the center portion of the top boundary; their temperature will stay fixed at `T_talik_base`, while the temperature of the other surface nodes will vary sinusoidally, representing the seasonal cycle

In [ ]:
# Create grid
grid = RasterModelGrid((nrows, ncols), dx)

# (if we're not wrapping boundaries)
grid.set_closed_boundaries_at_grid_edges(True, False, True, False)

# Create field(s)
T = grid.add_zeros("temperature", at="node")
q = grid.add_zeros("specific_heat_flux", at="link")

# Initialize
T = Ts_mean + Tgrad_mean * grid.y_of_node

# Set up lake and "tundra" nodes
lake_nodes = np.logical_and(
    grid.y_of_node == 0.0,
    np.logical_and(
        grid.x_of_node > 0.5 * lake_width, grid.x_of_node < 1.5 * lake_width
    ),
)
tundra_nodes = np.logical_and(
    grid.y_of_node == 0.0,
    lake_nodes is False,
)
T[lake_nodes] = T_talik_base

Plot the initial temperature field.

In [ ]:
fig, ax = plt.subplots()
imshow_grid(
    grid,
    T,
    cmap="coolwarm",
    vmin=-10.0,
    vmax=10.0,
    colorbar_label="Temperature (C)",
)
plt.title("Initial temperature field")
plt.xlabel("Distance (m)")
plt.ylabel("Depth (m)")
ax.invert_yaxis()

Next, run the time loop. The loop takes advantage of two of Landlab's numerical functions:

- `calc_grad_at_link` calculates the gradient in the given quantity (in this case temperature) between each adjacent pair of grid nodes. The resulting gradient values are mapped to the *links* that connect the node pairs. Every link has a direction as well as a magnitude. For example, link 0, which connects nodes 0 and 1, "points" in the direction of node 1. Node 1 is said to be the "head" of link 0, and node 0 is said to be the "tail" of link 0. Here a positive value of gradient means that temperature increases in the direction of the link; a negative value means temperature decreases in the direction of the link.

- `calc_flux_div_at_node` calculates the divergence of flux at each grid cell. The flux quantity (here representing heat flux per unit width) is a link-based array passed as an argument to the function. The function then adds up the total flux crossing each face of the grid cell, and divides the result by the cell area. To be consistent with the mathematical definition of divergence, the resulting flux divergence is positive when the net flux is outward, and negative when it is inward (into the cell). The result is then assigned to each cell's corresponding grid node. Nodes around the perimeter of the grid don't have cells, so these nodes are simply assigned zero. The function then returns an array of divergence values, where the length of the array is equal to the number of grid nodes.

The algorithm in the time loop does the following:

1. Update the surface temperature (outside the lake)
   
2. Calculate the temperature gradients at the links
   
3. Calculate heat fluxes at the "active" links (links along the side boundaries will have been flagged as "inactive", so this limitation to active links implements a no-flux condition along these boundaries)

4. Calculate the rate of change of temperature at the nodes

5. Update the temperature values at the "core" (i.e., non-boundary) nodes

In [ ]:
for i in range(num_steps):
    T[tundra_nodes] = Ts_mean + Ts_amp * np.sin(2.0 * np.pi * i * dt / period)
    Tgrad = grid.calc_grad_at_link(T)
    q[grid.active_links] = -conductivity * Tgrad[grid.active_links]
    dTdt = -one_over_rho_cp * grid.calc_flux_div_at_node(q)
    T[grid.core_nodes] += dTdt[grid.core_nodes] * dt

Plot the final temperature field:

In [ ]:
fig, ax = plt.subplots()
imshow_grid(
    grid,
    T,
    cmap="coolwarm",
    vmin=-10.0,
    vmax=10.0,
    colorbar_label="Temperature (C)",
)
plt.title("TEMPERATURE FIELD AROUND AN ARCTIC TALIK")
plt.xlabel("Distance (m)")
plt.ylabel("Depth (m)")
ax.invert_yaxis()

For more about Landlab, see [https://landlab.csdms.io](https://landlab.csdms.io).